In [ ]:
print("Hello World")

In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#data extraction from file
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf(data = 'Data/')

In [ ]:
#extracted_data

In [89]:
#splitting data into chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [90]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 4704


In [91]:
#text_chunks

In [92]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
import sys
print(sys.executable)

In [ ]:
embeddings = download_hugging_face_embeddings()


In [ ]:
query_result = embeddings.embed_query("hello world")
print("length", len(query_result))

In [ ]:
#query_result 

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')


In [93]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

#Create index only if it does NOT exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,   # must match your embedding model output size
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        )
    )


In [102]:
import sys
print(sys.executable)


c:\Users\HOME\anaconda3\envs\medibot\python.exe


In [103]:
import sys
!{sys.executable} -m pip install google-generativeai langchain-google-genai


In [104]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY


In [94]:
FIRST_RUN = False   # 👈 change this to True only once when adding docs

if FIRST_RUN:
    docsearch = PineconeVectorStore.from_documents(
        documents=text_chunks,
        embedding=embeddings,
        index_name=index_name
    )
else:
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )


In [95]:
#load the existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":8})


In [ ]:
retrieved_docs = retriever.invoke("What is acne?")

In [96]:
retrieved_docs

[Document(id='758f81af-b289-4434-b5e2-e0c3c9cd0ca0', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\Medical_book.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='3c7f630d-4cfa-44b3-a613-9dc57e8d2a91', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'modd

In [105]:
import os

# put your Gemini API key here (from Google AI Studio)
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY


In [106]:
import google.generativeai as genai
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

for m in genai.list_models():
    print(m.name, "→", m.supported_generation_methods)


models/embedding-gecko-001 → ['embedText', 'countTextTokens']
models/gemini-1.5-pro-latest → ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 → ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro → ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest → ['generateContent', 'countTokens']
models/gemini-1.5-flash → ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 → ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b → ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 → ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest → ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-preview-05-20 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateConten

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash-latest",   
    temperature=0.4,
    max_output_tokens=500
)



In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# ---- SYSTEM PROMPT ----
system_prompt = """
You are a medical assistant chatbot.

Rules:
1. If the document has relevant information, use it and say: "According to the provided document, ..."
2. If the document does not have relevant information but the user’s question is medical, answer using your own medical knowledge and say: "Based on general medical knowledge, ..."
3. If the question is not related to medicine, reply only with: "I don't know."
4. Always keep responses factual and concise.

{context}
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [97]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [98]:
docs = retriever.get_relevant_documents("What is acromegaly?")
for d in docs:
    print(d.page_content[:300])  # preview first 300 chars


Corticosteroids —A group of hormones produced
naturally by the adrenal gland or manufactured syn-
thetically. They are often used to treat inflammation.
Examples include cortisone and prednisone.
Immunocompromised —A state in which the
immune system is suppressed or not functioning
properly.
Meningi
cells and fibrin clot. Reagents are substances added
to the serum to produce a chemical reaction.
Thrombosis—Formation of a clot in the blood that
either blocks, or partially blocks a blood vessel.
The thrombus may lead to infarction, or death of
tissue, due to a blocked blood supply.
tors. Severe h
monly known coagulation disorder is hemophilia, a con-
dition in which patients bleed for long periods of time
before clotting. There are other coagulation disorders
with a variety of causes.
Description
Coagulation, or clotting, occurs as a complex
process involving several components of the blood.
appearance are also signs of V on Willebrand’s disease.
Hypoprothrombinemia
This disorder is a de

In [107]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Based on general medical knowledge, acne is a common skin condition that occurs when hair follicles become clogged with oil and dead skin cells.  This can lead to pimples, blackheads, and whiteheads.


In [ ]:
response = rag_chain.invoke({"input": "What is primie minister?"})
print(response["answer"])

In [101]:
response = rag_chain.invoke({"input": "What is acromegaly and gigantism?"})
print(response["answer"])

Based on general medical knowledge, acromegaly and gigantism are both caused by excessive growth hormone, but they occur at different ages.  Acromegaly occurs in adults after the growth plates have closed, resulting in enlargement of the hands, feet, and facial features. Gigantism occurs in children before the growth plates close, leading to excessive overall growth and increased height.
